In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/param-tunint-iot-b-2/sample_submission.csv
/kaggle/input/param-tunint-iot-b-2/train.csv
/kaggle/input/param-tunint-iot-b-2/test.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer

# Load datasets
train_df = pd.read_csv("/kaggle/input/param-tunint-iot-b-2/train.csv")
test_df = pd.read_csv("/kaggle/input/param-tunint-iot-b-2/test.csv")

# Drop 'id' and 'outcome' from train data
X = train_df.drop(columns=['id', 'outcome'])
y = train_df['outcome']

# Drop 'id' from test data
X_test = test_df.drop(columns=['id'])

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=["number"]).columns.tolist()

# OneHotEncode categorical columns and standardize numerical features
preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols),
    ('scaler', StandardScaler(), numerical_cols)
])

# Transform features
X = preprocessor.fit_transform(X)
X_test = preprocessor.transform(X_test)

# Encode target variable
y_encoded = pd.factorize(y)[0]

# Splitting data
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Model training and evaluation
models = {
    "RandomForest": RandomForestClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "LogisticRegression": LogisticRegression(max_iter=500),
    "XGBClassifier": XGBClassifier()
}

best_model = None
best_f1 = 0

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average='micro')
    print(f"{name} F1-Score: {f1:.4f}")
    if f1 > best_f1:
        best_f1 = f1
        best_model = model

# Hyperparameter tuning using GridSearchCV
param_grids = {
    "RandomForestClassifier": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10],
        "criterion": ["gini", "entropy"],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 2]
    },
    "GradientBoostingClassifier": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5],
        "subsample": [0.8, 1.0]
    },
    "LogisticRegression": {
        "C": [0.1, 1, 10],
        "solver": ["liblinear", "lbfgs"]
    },
    "XGBClassifier": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [3, 5],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0]
    }
}

if best_model:
    model_name = best_model.__class__.__name__
    if model_name in param_grids:
        print(f"Tuning {model_name} with GridSearchCV...")
        grid_search = GridSearchCV(best_model, param_grids[model_name], cv=5, scoring='f1_micro', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_params = grid_search.best_params_
        
        print(f"Best {model_name} parameters: {best_params}")
        print(f"Best {model_name} F1-Score after tuning: {grid_search.best_score_:.4f}\n")
        
        # Train the best model with full data
        best_model.set_params(**best_params)
        best_model.fit(X, y_encoded)

# Prediction on test data
test_predictions = best_model.predict(X_test)

# Convert numerical labels back to original labels
test_predictions = pd.Series(test_predictions).map(dict(enumerate(y.unique())))

# Save predictions to CSV
submission = pd.DataFrame({"id": test_df["id"], "outcome": test_predictions})
submission.to_csv("submission.csv", index=False)

print("Predictions saved to submission.csv!")

RandomForest F1-Score: 0.6802
GradientBoosting F1-Score: 0.6842
LogisticRegression F1-Score: 0.6761
XGBClassifier F1-Score: 0.7166
Tuning XGBClassifier with GridSearchCV...
Best XGBClassifier parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Best XGBClassifier F1-Score after tuning: 0.7389

Predictions saved to submission.csv!
